In [17]:
import pandas as pd
import numpy as np

data_rounded = pd.read_csv('input/data_rounded.csv')

# Remove na
data_rounded = data_rounded.dropna()
data_rounded.head()

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,40.03,-74.28,40.03,-74.28
1,40.05,-74.97,40.04,-74.96
2,40.05,-73.02,40.06,-72.99
3,40.06,-74.92,40.30,-74.84
4,40.06,-74.45,40.05,-74.45


In [18]:
len(data_rounded)

62886

In [19]:
import googlemaps

with open('/home/ec2-user/keys/gmap.txt', 'r') as f:
    gmap_key = str(f.read()).strip()
    
# Authenticate with google maps
gmaps = googlemaps.Client(key=gmap_key)

In [ ]:
from tqdm import tqdm_notebook

tqdm.pandas()

data_rounded['pickup'] = data_rounded['pickup_latitude'].astype(str) + "," + data_rounded['pickup_longitude'].astype(str)
data_rounded['dropoff'] = data_rounded['dropoff_latitude'].astype(str) + "," + data_rounded['dropoff_longitude'].astype(str)

def row_proc(pickup, dropoff):
    geocode_result = gmaps.distance_matrix(pickup,dropoff)
    #print (geocode_result)
    try:
        distance = float(geocode_result['rows'][0]['elements'][0]['distance']['text'].split()[0])
        duration = geocode_result['rows'][0]['elements'][0]['duration']['text'].split()
        if len(duration)==4:
            mins = float(duration[0])*60 + float(duration[2])
        else:
            mins = float(duration[0])
    except:
        mins = np.nan
        distance = np.nan
    return pd.Series((distance, mins))

data_rounded[['distance','duration']] = data_rounded.progress_apply(lambda row: row_proc(row.pickup, row.dropoff), axis=1)




  0%|          | 0/62886 [00:00<?, ?it/s]


  0%|          | 2/62886 [00:00<2:00:31,  8.70it/s]


  0%|          | 3/62886 [00:00<2:00:03,  8.73it/s]


  0%|          | 4/62886 [00:00<2:22:04,  7.38it/s]


  0%|          | 6/62886 [00:00<2:05:40,  8.34it/s]


  0%|          | 7/62886 [00:00<2:05:46,  8.33it/s]


  0%|          | 10/62886 [00:00<1:46:22,  9.85it/s]


  0%|          | 12/62886 [00:01<1:44:21, 10.04it/s]


  0%|          | 14/62886 [00:01<2:03:54,  8.46it/s]


  0%|          | 15/62886 [00:01<1:59:26,  8.77it/s]


  0%|          | 16/62886 [00:01<1:57:22,  8.93it/s]


  0%|          | 18/62886 [00:01<1:45:44,  9.91it/s]


  0%|          | 20/62886 [00:02<1:46:20,  9.85it/s]


  0%|          | 22/62886 [00:02<1:56:01,  9.03it/s]


  0%|          | 24/62886 [00:02<1:44:43, 10.00it/s]


  0%|          | 27/62886 [00:02<1:26:06, 12.17it/s]


  0%|          | 29/62886 [00:02<1:29:55, 11.65it/s]


  0%|          | 31/62886 [00:03<1:50:56,  9.44it/s]


  0%|          | 33/628

  0%|          | 219/62886 [00:23<1:39:55, 10.45it/s]


  0%|          | 222/62886 [00:23<1:27:32, 11.93it/s]


  0%|          | 224/62886 [00:23<1:24:08, 12.41it/s]


  0%|          | 226/62886 [00:23<1:25:02, 12.28it/s]


  0%|          | 228/62886 [00:23<1:44:02, 10.04it/s]


  0%|          | 230/62886 [00:24<2:02:08,  8.55it/s]


  0%|          | 231/62886 [00:24<2:07:29,  8.19it/s]


  0%|          | 232/62886 [00:24<2:25:08,  7.19it/s]


  0%|          | 233/62886 [00:24<2:39:41,  6.54it/s]


  0%|          | 236/62886 [00:24<2:03:16,  8.47it/s]


  0%|          | 239/62886 [00:24<1:38:02, 10.65it/s]


  0%|          | 241/62886 [00:24<1:27:37, 11.92it/s]


  0%|          | 243/62886 [00:25<1:38:41, 10.58it/s]


  0%|          | 245/62886 [00:25<1:42:58, 10.14it/s]


  0%|          | 247/62886 [00:25<1:49:30,  9.53it/s]


  0%|          | 249/62886 [00:25<1:56:16,  8.98it/s]


  0%|          | 251/62886 [00:26<1:45:52,  9.86it/s]


  0%|          | 253/62886 [00:26<1:57:50,  8.86

  1%|          | 482/62886 [00:47<2:30:14,  6.92it/s]


  1%|          | 483/62886 [00:47<2:20:03,  7.43it/s]


  1%|          | 484/62886 [00:47<2:23:15,  7.26it/s]


  1%|          | 487/62886 [00:47<1:57:39,  8.84it/s]


  1%|          | 490/62886 [00:48<1:34:17, 11.03it/s]


  1%|          | 492/62886 [00:48<1:47:17,  9.69it/s]


  1%|          | 494/62886 [00:48<2:02:42,  8.47it/s]


  1%|          | 496/62886 [00:48<1:54:23,  9.09it/s]


  1%|          | 498/62886 [00:49<1:55:25,  9.01it/s]


  1%|          | 500/62886 [00:49<1:56:34,  8.92it/s]


  1%|          | 501/62886 [00:49<1:57:26,  8.85it/s]


  1%|          | 502/62886 [00:49<2:03:54,  8.39it/s]


  1%|          | 503/62886 [00:49<2:01:22,  8.57it/s]


  1%|          | 504/62886 [00:49<2:03:04,  8.45it/s]


  1%|          | 505/62886 [00:49<2:02:16,  8.50it/s]


  1%|          | 506/62886 [00:50<2:16:40,  7.61it/s]


  1%|          | 507/62886 [00:50<2:22:34,  7.29it/s]


  1%|          | 509/62886 [00:50<2:13:03,  7.81

  1%|          | 648/62886 [01:07<2:01:51,  8.51it/s]


  1%|          | 651/62886 [01:07<1:39:23, 10.44it/s]


  1%|          | 654/62886 [01:07<1:33:33, 11.09it/s]


  1%|          | 656/62886 [01:08<1:41:00, 10.27it/s]


  1%|          | 658/62886 [01:08<1:36:09, 10.79it/s]


  1%|          | 660/62886 [01:08<1:33:48, 11.06it/s]


  1%|          | 662/62886 [01:08<1:37:13, 10.67it/s]


  1%|          | 665/62886 [01:08<1:29:04, 11.64it/s]


  1%|          | 667/62886 [01:09<1:27:31, 11.85it/s]


  1%|          | 669/62886 [01:09<1:52:06,  9.25it/s]


  1%|          | 672/62886 [01:09<1:42:16, 10.14it/s]


  1%|          | 674/62886 [01:09<1:49:14,  9.49it/s]


  1%|          | 676/62886 [01:10<2:04:06,  8.35it/s]


  1%|          | 677/62886 [01:10<2:05:41,  8.25it/s]


  1%|          | 678/62886 [01:10<2:08:34,  8.06it/s]


  1%|          | 679/62886 [01:10<2:08:55,  8.04it/s]


  1%|          | 680/62886 [01:10<2:11:25,  7.89it/s]


  1%|          | 682/62886 [01:10<2:09:19,  8.02

  1%|▏         | 857/62886 [01:30<2:00:41,  8.57it/s]


  1%|▏         | 858/62886 [01:31<1:58:44,  8.71it/s]


  1%|▏         | 859/62886 [01:31<2:18:55,  7.44it/s]


  1%|▏         | 860/62886 [01:31<2:15:29,  7.63it/s]


  1%|▏         | 861/62886 [01:31<2:18:44,  7.45it/s]


  1%|▏         | 862/62886 [01:31<2:21:31,  7.30it/s]


  1%|▏         | 863/62886 [01:31<2:36:46,  6.59it/s]


  1%|▏         | 864/62886 [01:31<2:28:21,  6.97it/s]


  1%|▏         | 865/62886 [01:32<2:43:32,  6.32it/s]


  1%|▏         | 866/62886 [01:32<2:27:48,  6.99it/s]


  1%|▏         | 868/62886 [01:32<2:04:19,  8.31it/s]


  1%|▏         | 871/62886 [01:32<1:47:51,  9.58it/s]


  1%|▏         | 873/62886 [01:32<1:52:12,  9.21it/s]


  1%|▏         | 875/62886 [01:33<1:55:19,  8.96it/s]


  1%|▏         | 876/62886 [01:33<1:55:34,  8.94it/s]


  1%|▏         | 878/62886 [01:33<1:54:21,  9.04it/s]


  1%|▏         | 879/62886 [01:33<2:25:53,  7.08it/s]


  1%|▏         | 880/62886 [01:33<2:17:28,  7.52

...

 37%|███▋      | 23002/62886 [37:22<1:10:15,  9.46it/s]


 37%|███▋      | 23004/62886 [37:23<1:07:00,  9.92it/s]


 37%|███▋      | 23007/62886 [37:23<57:38, 11.53it/s]  


 37%|███▋      | 23009/62886 [37:23<1:08:49,  9.66it/s]


 37%|███▋      | 23011/62886 [37:23<1:21:15,  8.18it/s]


 37%|███▋      | 23012/62886 [37:23<1:31:07,  7.29it/s]


 37%|███▋      | 23014/62886 [37:24<1:16:39,  8.67it/s]


 37%|███▋      | 23016/62886 [37:24<1:40:41,  6.60it/s]


 37%|███▋      | 23018/62886 [37:24<1:30:10,  7.37it/s]


 37%|███▋      | 23019/62886 [37:24<1:35:17,  6.97it/s]


 37%|███▋      | 23020/62886 [37:25<1:27:32,  7.59it/s]


 37%|███▋      | 23021/62886 [37:25<1:28:28,  7.51it/s]


 37%|███▋      | 23022/62886 [37:25<1:36:09,  6.91it/s]


 37%|███▋      | 23023/62886 [37:25<1:37:01,  6.85it/s]


 37%|███▋      | 23024/62886 [37:25<1:29:03,  7.46it/s]


 37%|███▋      | 23025/62886 [37:25<1:22:26,  8.06it/s]


 37%|███▋      | 23027/62886 [37:25<1:08:33,  9.69it/s]


 37%|███▋     

 37%|███▋      | 23260/62886 [37:45<54:26, 12.13it/s]


 37%|███▋      | 23262/62886 [37:45<51:23, 12.85it/s]


 37%|███▋      | 23265/62886 [37:45<43:14, 15.27it/s]


 37%|███▋      | 23267/62886 [37:45<45:17, 14.58it/s]


 37%|███▋      | 23269/62886 [37:45<48:09, 13.71it/s]


 37%|███▋      | 23271/62886 [37:46<48:40, 13.57it/s]


 37%|███▋      | 23273/62886 [37:46<46:55, 14.07it/s]


 37%|███▋      | 23275/62886 [37:46<42:51, 15.40it/s]


 37%|███▋      | 23277/62886 [37:46<44:45, 14.75it/s]


 37%|███▋      | 23279/62886 [37:46<43:43, 15.10it/s]


 37%|███▋      | 23281/62886 [37:46<47:54, 13.78it/s]


 37%|███▋      | 23283/62886 [37:46<53:31, 12.33it/s]


 37%|███▋      | 23286/62886 [37:47<49:38, 13.30it/s]


 37%|███▋      | 23289/62886 [37:47<45:55, 14.37it/s]


 37%|███▋      | 23291/62886 [37:47<49:40, 13.28it/s]


 37%|███▋      | 23293/62886 [37:47<48:50, 13.51it/s]


 37%|███▋      | 23295/62886 [37:47<47:24, 13.92it/s]


 37%|███▋      | 23297/62886 [37:47<52:38, 12.53

 37%|███▋      | 23517/62886 [38:06<1:30:43,  7.23it/s]


 37%|███▋      | 23518/62886 [38:06<1:31:40,  7.16it/s]


 37%|███▋      | 23519/62886 [38:06<1:38:55,  6.63it/s]


 37%|███▋      | 23521/62886 [38:06<1:32:26,  7.10it/s]


 37%|███▋      | 23522/62886 [38:06<1:27:23,  7.51it/s]


 37%|███▋      | 23524/62886 [38:07<1:26:25,  7.59it/s]


 37%|███▋      | 23527/62886 [38:07<1:13:53,  8.88it/s]


 37%|███▋      | 23528/62886 [38:07<1:14:29,  8.81it/s]


 37%|███▋      | 23529/62886 [38:07<1:14:31,  8.80it/s]


 37%|███▋      | 23530/62886 [38:07<1:16:55,  8.53it/s]


 37%|███▋      | 23532/62886 [38:07<1:10:31,  9.30it/s]


 37%|███▋      | 23533/62886 [38:07<1:16:44,  8.55it/s]


 37%|███▋      | 23534/62886 [38:08<1:29:21,  7.34it/s]


 37%|███▋      | 23535/62886 [38:08<1:29:10,  7.35it/s]


 37%|███▋      | 23536/62886 [38:08<1:24:10,  7.79it/s]


 37%|███▋      | 23538/62886 [38:08<1:22:38,  7.94it/s]


 37%|███▋      | 23539/62886 [38:08<1:23:44,  7.83it/s]


 37%|███▋     

 38%|███▊      | 23748/62886 [38:27<1:36:56,  6.73it/s]


 38%|███▊      | 23750/62886 [38:28<1:24:57,  7.68it/s]


 38%|███▊      | 23752/62886 [38:28<1:20:27,  8.11it/s]


 38%|███▊      | 23753/62886 [38:28<1:22:06,  7.94it/s]


 38%|███▊      | 23754/62886 [38:28<1:27:15,  7.48it/s]


 38%|███▊      | 23755/62886 [38:28<1:23:06,  7.85it/s]


 38%|███▊      | 23756/62886 [38:28<1:24:27,  7.72it/s]


 38%|███▊      | 23757/62886 [38:29<1:37:07,  6.72it/s]


 38%|███▊      | 23758/62886 [38:29<1:35:03,  6.86it/s]


 38%|███▊      | 23760/62886 [38:29<1:32:16,  7.07it/s]


 38%|███▊      | 23761/62886 [38:29<1:29:55,  7.25it/s]


 38%|███▊      | 23762/62886 [38:29<1:35:42,  6.81it/s]


 38%|███▊      | 23763/62886 [38:29<1:28:32,  7.36it/s]


 38%|███▊      | 23764/62886 [38:29<1:34:15,  6.92it/s]


 38%|███▊      | 23765/62886 [38:30<1:36:26,  6.76it/s]


 38%|███▊      | 23766/62886 [38:30<1:41:47,  6.40it/s]


 38%|███▊      | 23767/62886 [38:30<1:35:55,  6.80it/s]


 38%|███▊     

 38%|███▊      | 23896/62886 [38:47<1:18:28,  8.28it/s]


 38%|███▊      | 23897/62886 [38:47<1:28:22,  7.35it/s]


 38%|███▊      | 23900/62886 [38:47<1:14:51,  8.68it/s]


 38%|███▊      | 23903/62886 [38:47<1:06:51,  9.72it/s]


 38%|███▊      | 23905/62886 [38:48<1:10:00,  9.28it/s]


 38%|███▊      | 23907/62886 [38:48<1:14:20,  8.74it/s]


 38%|███▊      | 23908/62886 [38:48<1:13:39,  8.82it/s]


 38%|███▊      | 23909/62886 [38:48<1:13:37,  8.82it/s]


 38%|███▊      | 23910/62886 [38:48<1:11:27,  9.09it/s]


 38%|███▊      | 23911/62886 [38:48<1:27:18,  7.44it/s]


 38%|███▊      | 23912/62886 [38:48<1:25:34,  7.59it/s]


 38%|███▊      | 23913/62886 [38:49<1:25:29,  7.60it/s]


 38%|███▊      | 23914/62886 [38:49<1:32:58,  6.99it/s]


 38%|███▊      | 23916/62886 [38:49<1:25:18,  7.61it/s]


 38%|███▊      | 23917/62886 [38:49<1:33:46,  6.93it/s]


 38%|███▊      | 23918/62886 [38:49<1:26:54,  7.47it/s]


 38%|███▊      | 23920/62886 [38:49<1:21:02,  8.01it/s]


 38%|███▊     

In [ ]:
data_rounded.to_csv('input/rounded_distances.csv', index = False)